In [1]:
## Step 1 - Import all required libraries. All of these are pre- installed by MOSAIC AI 
import numpy as np
import pandas as pd


# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
#from mpl_toolkits.basemap import Basemap
#import seaborn as sns


# Prediction
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import itertools

# Classifiers
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier # Validation

# Data prepping
from sklearn.preprocessing import LabelEncoder

print("Libraries imported.")

Libraries imported.


In [2]:
### Step 2 - Connect to Catalog API to gfetch the data prepared in DataPrep section LENS - GTD_preparation_ML
#from connector.mosaicio import MosaicioConnector
#ioConnector = MosaicioConnector()
#data = ioConnector.getPandasDataFrame(name="GTD_preparation_ML_550",row_count=10000000)
#data = ioConnector.getPandasDataFrame(name="GTD_preparation_ML_903",row_count=10000000)

In [3]:
data=pd.read_csv("/data/GTD_data (1).csv", error_bad_lines=False)

## 2.2. Data selection and description
The main characteristics of the dataset with respect to the research questions are researched in the exploratory data analysis. These characteristics, for instance trends, potential causality or correlation found, will be used to predict future events.


# 4. Predictive analysis

## Can we predict if there will be multiple attacks from GTD-exclusive features?
Now, columns that are exclusively part of the Global Terrorism Database will be tested for their significance on predicting casualties. 


In [4]:
data.columns

Index(['Unnamed: 0', 'country', 'claimed', 'nkill', 'resolution', 'target1',
       'attacktype1', 'nwound', 'nkillus', 'attack_typr_txt_3', 'nwoundus',
       'attack_typr_txt_1', 'nkillter', 'property', 'longitude', 'iday',
       'iyear', 'suicide', 'gname', 'nationality', 'success', 'provstate',
       'propvalue', 'region', 'eventid', 'gsubname', 'nwoundte', 'country_txt',
       'region_txt', 'city', 'latitude', 'weapon_type', 'claimmode',
       'extended', 'alternative_txt', 'motive', 'imonth', 'specificity',
       'corp1', 'target_type', 'propextent', 'multi_attack_flag', 'location',
       'target_sub_type', 'ncasualties', 'weapon_type_c', 'vicinity_city',
       'has_casualities'],
      dtype='object')

In [5]:
#  data.loc[:,feature_cols]

In [6]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split
feature_cols = [
                #'iyear', 
                'imonth', 
                #'iday',
                #'latitude', 'longitude',
    
                'extended',
                'vicinity_city',
                'nkill', 'nwound',
                'success',
                'suicide',
                'claimed',
                'property',
    
                #'country_txt',
                'region_txt',
                'attack_typr_txt_1',
                #'nationality',
                'gname',
                'target_type',
                'weapon_type_c',
]
target_col = 'multi_attack_flag'
data["multi_attack_flag"] = data["multi_attack_flag"].astype(int)

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle
 
#df_train = pd.DataFrame({'names': ['Jack', 'Helen', 'Nick', 'Helen'],'jobs': ['Manager', 'HR', 'DevOps', 'HR']})
 
#df_test = pd.DataFrame({'names': [ 'Helen', 'Nick', 'Helen'],'jobs': ['HR', 'DevOps', 'HR']})
 
#df_deploy = pd.DataFrame({'names': [ 'Helen', 'Nick'],'jobs': ['HR', 'DevOps']})

#data = data.loc[:, feature_cols] 
categorical_features = data.loc[:,feature_cols].select_dtypes(include=[object]).columns.values.tolist()
####------------Create Dictionary----------####
le = LabelEncoder()
dict_all = dict(zip([], []))

for col in categorical_features:
    temp_keys = data[col].values
    temp_values = le.fit_transform(data[col])
    dict_temp = dict(zip(temp_keys, temp_values))
    dict_all[col] = dict_temp


In [8]:
data=data[categorical_features].copy()

In [9]:
# data[categorical_features].to_csv("/data/test_GTD_data.csv")

In [8]:
import time

In [9]:
categorical_features

['region_txt', 'attack_typr_txt_1', 'gname', 'target_type', 'weapon_type_c']

In [ ]:
start = time.time()
categorical_features = ['gname']


for col in categorical_features:
    print(col)
    data.replace(dict_all[col], inplace=True)

# sleeping for 1 sec to get 10 sec runtime
time.sleep(1)

# end time
end = time.time()

gname


In [ ]:
t2=end-start
t2
#16.584686279296875

#2.5133562088012695 otherwise

In [12]:
data.shape
#(181691, 48)

(181691, 5)

In [ ]:
##data[categorical_features].apply(lambda x: np.nan if x.startswith('XXX') else x)

In [ ]:
# Define X and Y.
X = data[feature_cols].fillna(0)
data[target_col] = data[target_col].astype('category',copy=False)
y = data[target_col].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=20,
                              random_state=0)

forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
fnames = [feature_cols[i] for i in indices]

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), fnames, rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
# Better validation with K-fold.
model = RandomForestClassifier(n_estimators=20)

scores = cross_val_score(model, X, y, cv=10) # Ten-fold cross validation.
print(scores)
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

In [ ]:
model = RandomForestClassifier(n_estimators=20)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

np.mean(y_pred == y_test)

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
 

def plot_confusion_matrix(cm,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
 
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
 
# Compute confusion matrix
np.set_printoptions(precision=2)
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix,
                      title='Confusion matrix, without normalization')
 
# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, normalize=True,
                      title='Normalized confusion matrix')
 
plt.show()

In [ ]:
import pandas as pd
from mosaicml import *
from mosaicml.constants import MLModelFlavours
import warnings

@scoring_func
def predict_multiattacks(model, request):
    global dict_all
    data_dict = request.json["payload"]
    df1=pd.DataFrame(columns=data_dict.keys())
    df1 = df1.append(data_dict, ignore_index=True)
    categorical_features = df1.select_dtypes(include=[object]).columns.values.tolist()
    for col in categorical_features:
        df1.replace(dict_all[col], inplace=True)
    out = model.predict(df1)    
    return str(out[0])

In [ ]:
Y_true = y_test

Y_pred = model.predict(X_test)
features = list(X_train)

In [ ]:
### new register model
register_model(
    model,
    predict_multiattacks,
    "predict_multiple_attacks",
    "predict_more_attacks_coming",
    MLModelFlavours.sklearn,
    init_script="mkdir abc \\n cd abc \\n touch abc.txt",
    input_type="json",
    explain_ai=True,
    x_train=np.array(X_train), 
    y_train=np.array(y_train), 
    y_true=Y_true,
    y_pred=Y_pred,
    feature_names=features,
    feature_ids=features, 
    target_names=["no multiple attack", "multiple attack"],
    model_type="classification",
    prob = model.predict_proba,
)

In [4]:
data

,Unnamed: 0,country,claimed,nkill,resolution,target1,attacktype1,nwound,nkillus,attack_typr_txt_3,...,corp1,target_type,propextent,multi_attack_flag,location,target_sub_type,ncasualties,weapon_type_c,vicinity_city,has_casualities
0,0,45,NaN,NaN,2021-07-06 10:33:00,NaN,9,NaN,NaN,NaN,...,NaN,Business,3.0,False,NaN,Retail/Grocery/Bakery,1,Unknown,0,1
1,1,95,0.0,1.0,2021-07-06 10:33:00,A roadside bomb was used in the attack.,3,3.0,0.0,NaN,...,Iraqi Army,Military,4.0,False,The incident occurred in the Albu Aythah neigh...,Military Transportation/Vehicle (excluding con...,4,Explosives,0,1
2,2,95,0.0,1.0,2021-07-06 10:33:00,An explosives-laden belt was used in the attack.,3,0.0,0.0,NaN,...,Unknown,Unknown,NaN,False,The incident occurred in the Ad Darwish area.,NaN,1,Explosives,0,1
3,3,95,0.0,2.0,2021-07-06 10:33:00,NaN,3,7.0,0.0,NaN,...,Unknown,Private Citizens & Property,NaN,False,NaN,Marketplace/Plaza/Square,9,Explosives,0,1
4,4,95,1.0,6.0,2021-07-06 10:33:00,An explosive-laden vest was used in the attack.,3,17.0,0.0,NaN,...,Iraqi Armed Forces,Military,NaN,False,The incident occurred in the Hayy Al Basatin a...,Military Checkpoint,23,Explosives,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,181686,153,0.0,0.0,2021-07-06 10:33:35,Improvised explosive devices were used in the ...,3,0.0,0.0,NaN,...,NaN,Utilities,3.0,False,In the Chethar sub-division of Dera Murad Jama...,Electricity,0,Explosives,0,0
181687,181687,153,0.0,0.0,2021-07-06 10:33:35,An improvised explosive device was used in the...,3,0.0,0.0,NaN,...,NaN,Utilities,3.0,False,NaN,Gas,0,Explosives,0,0
181688,181688,153,0.0,0.0,2021-07-06 10:33:35,Several improvised explosive devices were used...,3,0.0,0.0,NaN,...,Wana Press Club,Journalists & Media,3.0,False,NaN,Newspaper Journalist/Staff/Facility,0,Explosives,0,0
181689,181689,4,0.0,0.0,2009-02-19 00:00:00,It is unknown if weapons were used in the attack.,6,0.0,0.0,NaN,...,NaN,Private Citizens & Property,NaN,False,The attack took place in Musahi.,Laborer (General)/Occupation Identified,0,Unknown,0,0


In [5]:
pwd

'/notebooks/notebooks'

In [9]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.21.3.


ModuleNotFoundError: No module named 'Crypto'

In [1]:
import Crypto
import Cryptodome
import Cython
import OpenSSL
import PIL
import PyQt5
import SPARQLWrapper
import abc
import adodbapi
import afxres
import aifc
import alabaster
import antigravity
import anytree
import argparse
import array
import asgiref
import asn1crypto
import ast
import astroid
import astropy
import asynchat
import asyncio
import asyncore
import atexit
import atomicwrites
import attr
import audioop
import autograd
import autoreload
import babel
import backcall
import backports
import base64
import bdb
import billiard
import binascii
import binhex
import binstar_client
import bisect
import bitarray
import bkcharts
import bleach
import bokeh
import boto
import boto3
import botocore
import bottleneck
import brain_argparse
import brain_attrs
import brain_builtin_inference
import brain_collections
import brain_crypt
import brain_curses
import brain_dataclasses
import brain_dateutil
import brain_fstrings
import brain_functools
import brain_gi
import brain_hashlib
import brain_http
import brain_io
import brain_mechanize
import brain_multiprocessing
import brain_namedtuple_enum
import brain_nose
import brain_numpy_core_fromnumeric
import brain_numpy_core_function_base
import brain_numpy_core_multiarray
import brain_numpy_core_numeric
import brain_numpy_core_numerictypes
import brain_numpy_core_umath
import brain_numpy_ndarray
import brain_numpy_random_mtrand
import brain_numpy_utils
import brain_pkg_resources
import brain_pytest
import brain_qt
import brain_random
import brain_re
import brain_six
import brain_ssl
import brain_subprocess
import brain_threading
import brain_typing
import brain_uuid
import bs4
import builtins
import bz2
import cProfile
import calendar
import cartopy
import casadi
import certifi
import cffi
import cgi
import cgitb
import chardet
import chunk
import click
import cloudpickle
import clyent
import cmath
import cmd
import coala_utils
import code
import codecs
import codeop
import collections
import colorama
import colorcet
import colorlog
import colorsys
import commctrl
import compileall
import comtypes
import concurrent
import contextlib
import contextlib2
import contextvars
import copy
import copyreg
import crypt
import cryptography
import cssselect
import cycler
import cython
import cythonmagic
import cytoolz
import damona
import dask
import dataclasses
import datacompy
import datashader
import datashape
import datetime
import dateutil
import dbi
import dbm
import dde
import decimal
import decorator
import defusedxml
import deprecated
import difflib
import dill
import dis
import distributed
import distutils
import django
import docs
import doctest
import docutils
import dummy_threading
import easy_install
import easydev
import editdistance
import email
import encodings
import ensurepip
import entrypoints
import enum
import errno
import et_xmlfile
import fastcache
import faulthandler
import feedfinder2
import feedparser
import filecmp
import fileinput
import filelock
import fitter
import flask
import fnmatch
import formatter
import fractions
import fsspec
import ftplib
import functools
import future
import fuzzy_matcher
import fuzzywuzzy
import gc
import genericpath
import geograpy
import geoviews
import getopt
import getpass
import gettext
import gevent
import glob
import glob2
import greenlet
import gzip
import h5py
import hashlib
import heapdict
import heapq
import hmac
import holoviews
import html
import html5lib
import http
import idlelib
import idna
import image
import imageio
import imagesize
import imaplib
import imghdr
import imp
import importlib
import importlib_metadata
import inspect
import io
import ipaddress
import ipykernel
import ipykernel_launcher
import ipython_genutils
import ipywidgets
import isapi
import isodate
import isort
import isympy
import itertools
import itsdangerous
import jdcal
import jedi
import jellyfish
import jieba
import jinja2
import jmespath
import joblib
import json
import json5
import jsonschema
import jwt
import keras
import keyring
import keyword
import kiwisolver
import lazy_object_proxy
import lib2to3
import libarchive
import libfuturize
import libpasteurize
import lief
import lifetimes
import linecache
import llvmlite
import locale
import locket
import lodstorage
import logging
import lxml
import lzma
import macpath
import mailbox
import mailcap
import markdown
import markupsafe
import marshal
import math
import matplotlib
import mccabe
import mcerp
import melting
import menuinst
import meshio
import mimetypes
import mistune
import mkl
import mkl_fft
import mkl_random
import mlxtend
import mmap
import mmapfile
import mmsystem
import mock
import modulefinder
import more_itertools
import mpmath
import msgpack
import msilib
import msrest
import msvcrt
import multipledispatch
import multiprocessing
import navigator_updater
import nbconvert
import nbformat
import netbios
import netrc
import networkx
import newspaper
import nltk
import nntplib
import nose
import notebook
import nt
import ntpath
import ntsecuritycon
import nturl2path
import numba
import numbergen
import numbers
import numexpr
import numpy
import numpydoc
import oauthlib
import odbc
import olefile
import opcode
import openpyxl
import operator
import optparse
import os
import oscrypto
import owslib
import packaging
import pandas
import pandocfilters
import panel
import param
import parser
import parso
import partd
import past
import path
import pathlib
import pathlib2
import patsy
import pdb
import pep8
import perfmon
import pexpect
import pickle
import pickleshare
import pickletools
import pip
import pipes
import pkg_resources
import pkginfo
import pkgutil
import platform
import plistlib
import plotly
import plotlywidget
import pluggy
import ply
import poplib
import posixpath
import pprint
import profile
import prometheus_client
import prompt_toolkit
import pstats
import psutil
import pty
import ptyprocess
import pvectorc
import py
import py_compile
import pybamm
import pyclbr
import pycodestyle
import pycosat
import pycountry
import pycparser
import pyct
import pycurl
import pydoc
import pydoc_data
import pyepsg
import pyexpat
import pyflakes
import pygments
import pykdtree
import pylab
import pylint
import pymisp
import pyodbc
import pyparsing
import pyproj
import pyreadline
import pyrsistent
import pytest
import pytest_arraydiff
import pytest_doctestplus
import pytest_openfiles
import pytest_remotedata
import pythoncom
import pytils
import pytz
import pyviz_comms
import pywin
import pywin32_testutil
import pywintypes
import pywt
import pyximport
import qtawesome
import qtconsole
import qtpy
import queue
import quopri
import random
import rasutil
import rdflib
import re
import readline
import regcheck
import regutil
import reprlib
import requests
import requests_file
import requests_oauthlib
import retrying
import rlcompleter
import rmagic
import rope
import ruamel_yaml
import run
import runpy
import s3transfer
import sched
import scipy
import script
import scripts
import seaborn
import secrets
import select
import selectors
import semver
import send2trash
import servicemanager
import setuptools
import sgmllib
import shapefile
import shapely
import shelve
import shlex
import shutil
import signal
import simplegeneric
import singledispatch
import singledispatch_helpers
import sip
import sipconfig
import sipdistutils
import site
import six
import skfem
import skimage
import sklearn
import smtpd
import smtplib
import sndhdr
import snowballstemmer
import socket
import socketserver
import socks
import sockshandler
import sortedcollections
import sortedcontainers
import soupsieve
import sphinx
import sphinxcontrib
import spotipy
import sqlalchemy
import sqlite3
import sqlparse
import sre_compile
import sre_constants
import sre_parse
import ssl
import sspi
import sspicon
import stat
import statistics
import statsmodels
import storemagic
import string
import stringprep
import struct
import subprocess
import sunau
import symbol
import sympy
import sympyprinting
import symtable
import sys
import sysconfig
import tables
import tabnanny
import tarfile
import tblib
import telnetlib
import tempfile
import terminado
import test
import test_data
import test_pycosat
import testpath
import tests
import textblob
import textwrap
import this
import threading
import time
import timeit
import timer
import tinysegmenter
import tkinter
import tldextract
import tlz
import token
import tokenize
import tools
import toolz
import tornado
import tqdm
import trace
import traceback
import tracemalloc
import traitlets
import tty
import turtle
import turtledemo
import tweepy
import twitterscraper
import types
import typing
import typing_extensions
import unicodecsv
import unicodedata
import unittest
import urllib
import urllib3
import uu
import uuid
import venv
import warnings
import wave
import wcwidth
import weakref
import webbrowser
import webencodings
import werkzeug
import wheel
import widgetsnbextension
import wordcloud
import wrapt
import wsgiref
import wx
import xarray
import xdrlib
import xlrd
import xlsxwriter
import xlwings
import xlwt
import xml
import xmlrpc
import xxsubtype
import zict
import zipapp
import zipfile
import zipimport
import zipp
import zlib
import zmq

ModuleNotFoundError: No module named 'Crypto'